## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-26-14-41-55 +0000,nyt,President Oversees Signing of Cambodia-Thailan...,https://www.nytimes.com/live/2025/10/25/us/tru...
1,2025-10-26-14-41-53 +0000,nyt,Trump Leans Into Showmanship on Day 1 of His A...,https://www.nytimes.com/2025/10/25/world/asia/...
2,2025-10-26-14-39-33 +0000,nypost,Potential 2028 hopeful Pete Buttigieg slams Cu...,https://nypost.com/2025/10/26/us-news/potentia...
3,2025-10-26-14-37-34 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...
4,2025-10-26-14-35-00 +0000,wsj,"U.S., China Sound Confident Note After Trade T...",https://www.wsj.com/world/china/bessent-sounds...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,37
40,louvre,11
118,heist,10
11,asia,9
130,he,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-10-26-14-37-34 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...,75
88,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...,75
155,2025-10-25-20-30-41 +0000,nypost,Trump not meeting with Putin again until he kn...,https://nypost.com/2025/10/25/world-news/trump...,71
13,2025-10-26-13-39-19 +0000,bbc,Will Trump's 'peace deal' solve Thailand-Cambo...,https://www.bbc.com/news/videos/cy40n3ykx93o?a...,70
106,2025-10-26-01-51-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...,70


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,75,2025-10-26-14-37-34 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...
61,52,2025-10-26-09-47-45 +0000,nypost,2 Louvre robbery suspects busted in $100M muse...,https://nypost.com/2025/10/26/world-news/louvr...
75,35,2025-10-26-09-00-00 +0000,wsj,Rep. Jared Golden didn’t just side with the GO...,https://www.wsj.com/politics/policy/jared-gold...
92,35,2025-10-26-05-21-38 +0000,nypost,Bikini-clad Brazilian influencer Melissa Said ...,https://nypost.com/2025/10/26/world-news/brazi...
127,34,2025-10-25-23-10-24 +0000,nypost,NYC voters run to the polls on first day of ea...,https://nypost.com/2025/10/25/us-news/nyc-vote...
106,33,2025-10-26-01-51-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...
88,29,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...
98,26,2025-10-26-03-08-19 +0000,nypost,Sister of slain NJ man blasts cop who allegedl...,https://nypost.com/2025/10/25/us-news/sister-o...
161,25,2025-10-25-18-55-14 +0000,nyt,Canada-U.S. Tensions Stay on Baseball Field Du...,https://www.nytimes.com/2025/10/25/sports/base...
32,23,2025-10-26-11-47-44 +0000,nypost,"London police nab sex offender, asylum-seeker ...",https://nypost.com/2025/10/26/world-news/londo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
